In [ ]:
! pip install requirements.txt

In [ ]:
import image_grid

: 

In [ ]:
PADDING = 50 # padding between images in pixels
IMGDIR = "./imgs" # folder containing the images
ROTATION = 180 # rotate all images before placing
STANDARDIZED = True # True if all images are same size, False otherwise

In [ ]:
IMGS = get_imgs(IMGDIR) # get list of images from IMGDIR

In [ ]:
ROWS = 0
COLUMNS = 0
DIMS = (ROWS, COLUMNS) # grid dimensions

In [ ]:
combine_images(PADDING, IMGS, IMGDIR, DIMS, ROTATION, STANDARDIZED)